In [1]:
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import time


In [30]:
Data =pd.read_csv("housing.csv")
X = Data.drop(['median_house_value'] , axis=1)
Y = Data['median_house_value']
Y = Y.values.reshape(-1,1)

X



,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,NEAR BAY
...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,INLAND


In [ ]:

Place=['INLAND','ISLAND','NEAR BAY','<1H OCEAN','NEAR OCEAN']

encoder = OrdinalEncoder(
    categories=[Place],
    handle_unknown='use_encoded_value',
    unknown_value=-1
)
cat_cols = ['ocean_proximity']
new_cols = ['Place']
encoder.fit(X[cat_cols])

X[new_cols] = encoder.transform(X[cat_cols])

X = X.select_dtypes(include=['number'])

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15,random_state=42)

median_value_train = X_train.median(numeric_only = True)
X_train = X_train.fillna(median_value_train)

median_value_test =  X_test.median(numeric_only = True)
X_test = X_test.fillna(median_value_test)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
y_scaler = StandardScaler()
y_scaler.fit(Y_train)
Y_train = y_scaler.transform(Y_train)


In [17]:
X_train_tensor = torch.tensor(X_train,dtype = torch.float32)
X_test_tensor = torch.tensor(X_test,dtype = torch.float32)
Y_train_tensor = torch.tensor(Y_train,dtype = torch.float32).view(-1,1)
Y_test_tensor = torch.tensor(Y_test,dtype = torch.float32).view(-1,1)
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)
train_loader = DataLoader(train_dataset,batch_size=128,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=128)

In [24]:
NUM_EPOCHS =500
RANDOM_SEED = 42
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [28]:
class Model(torch.nn.Module):
    def __init__(self,num_features):
        super().__init__()
        self.my_net = torch.nn.Sequential(
            torch.nn.Linear(in_features=num_features, out_features=6),
            torch.nn.BatchNorm1d(num_features=6),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=6, out_features=4),
            torch.nn.BatchNorm1d(num_features=4),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=4, out_features=3),
            torch.nn.BatchNorm1d(num_features=3),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=3, out_features=1),


        )
    def forward(self, x):
        out = self.my_net(x)
        return out


torch.manual_seed(RANDOM_SEED)
model = Model(num_features=9)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

def compute_loss(net,dataloader):
    curr_loss = 0
    with torch.no_grad():
        for cnt , (fet, output) in enumerate(dataloader):
            fet = fet.view((-1, 9)).to(DEVICE)
            output = output.view((-1,1)).to(DEVICE)
            out = net(fet)
            loss = F.mse_loss(out,output)
            curr_loss += loss
        return float(curr_loss)/len(dataloader)

start_time = time.time()
minibatch_cost = []
epoch_cost = []

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx , (features , targets) in enumerate(train_loader):
        features = features.view(-1,9).to(DEVICE)
        targets = targets.to(DEVICE)
        pred = model.forward(features)
        cost = F.mse_loss(pred,targets.float().view(-1,1))
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        minibatch_cost.append(cost.item())
        if not batch_idx % 64:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f'
                   %(epoch+1, NUM_EPOCHS, batch_idx,
                     len(train_loader), cost.item()))
    cost = compute_loss(model, train_loader)
    epoch_cost.append(cost)
    print('Epoch: %03d/%03d Train Cost: %.4f' % (
            epoch+1, NUM_EPOCHS, cost))
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))

print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))











Epoch: 001/500 | Batch 000/138 | Cost: 0.9603
Epoch: 001/500 | Batch 064/138 | Cost: 1.1225
Epoch: 001/500 | Batch 128/138 | Cost: 0.8162
Epoch: 001/500 Train Cost: 0.9055
Time elapsed: 0.01 min
Epoch: 002/500 | Batch 000/138 | Cost: 0.5747
Epoch: 002/500 | Batch 064/138 | Cost: 0.8272
Epoch: 002/500 | Batch 128/138 | Cost: 0.7444
Epoch: 002/500 Train Cost: 0.5564
Time elapsed: 0.02 min
Epoch: 003/500 | Batch 000/138 | Cost: 0.5643
Epoch: 003/500 | Batch 064/138 | Cost: 0.5509
Epoch: 003/500 | Batch 128/138 | Cost: 0.3420
Epoch: 003/500 Train Cost: 0.3929
Time elapsed: 0.03 min
Epoch: 004/500 | Batch 000/138 | Cost: 0.3019
Epoch: 004/500 | Batch 064/138 | Cost: 0.2987
Epoch: 004/500 | Batch 128/138 | Cost: 0.3619
Epoch: 004/500 Train Cost: 0.3476
Time elapsed: 0.03 min
Epoch: 005/500 | Batch 000/138 | Cost: 0.3320
Epoch: 005/500 | Batch 064/138 | Cost: 0.4188
Epoch: 005/500 | Batch 128/138 | Cost: 0.4298
Epoch: 005/500 Train Cost: 0.3326
Time elapsed: 0.04 min
Epoch: 006/500 | Batch 00

KeyboardInterrupt: 